In [1]:
import os
import re
import import_ipynb
import numpy as np
import pandas as pd
from utils import load_sequence,replace_base
from tqdm import tqdm
from functools import reduce


importing Jupyter notebook from utils.ipynb


In [2]:
#定义训练集和测试集的目录
training_set_path='../data/Training Set'
testing_set_path='../data/Testing Set'

class sequence_df:
    def __init__(self,seq_set,chunk_size):
        self.seq_set=seq_set;
        self.chunk_size=chunk_size;
        self.half_ck=int((chunk_size-1) / 2);
        self.base_matrix=np.eye(4,dtype=np.int);
        self.df=''
    
    #编码特征序列
    def feature_code(self,seqs):
        seqs=seqs.replace('a','0').replace('c','1').replace('t','2').replace('g','3');
        #print("seqs are {}".format(seqs));
        seqs_code=np.concatenate([self.base_matrix[int(i)] for i in seqs]);
        return seqs_code;
   #过滤自身错码碱基
    def filter_base(self,seqs):
        match_base=re.compile(pattern='[a-z]');
        seqs=[seq for seq in seqs if len(seq)==self.chunk_size and match_base.findall(string=replace_base(seq))== [] and replace_base(seq)!=""]
        return seqs;
        
    #提取特征值:将Accept_sites Donor_sites 用列表的形式取出来
    def feature_df(self):
        seq_df=load_sequence(self.seq_set);
        seq_df['Accept_sites']=seq_df['ADS'].apply(lambda x:{int(i.split('..')[0]) for i in x.split(',')})
        seq_df['Donor_sites']=seq_df['ADS'].apply(lambda x:{int(i.split('..')[1]) for i in x.split(',')})
        seq_df['Ordinary_sites']=seq_df.apply(lambda x:set(range(self.half_ck+1,len(x['sequence'])-self.half_ck))-x['Accept_sites']-x['Donor_sites'],axis=1);
        #获取Accept,Donor,Ordinary的sequence的序列
        tqdm.pandas(desc="processing")
        seq_df['Donor_seqs']=seq_df.progress_apply(lambda x:np.array([x['sequence'][i-self.half_ck:i+self.half_ck+1] for i in x['Donor_sites']]),axis=1);
        seq_df['Accept_seqs']=seq_df.progress_apply(lambda x:np.array([x['sequence'][i-self.half_ck:i+self.half_ck+1] for i in x['Accept_sites']]),axis=1);
        seq_df['Ordinary_seqs']=seq_df.progress_apply(lambda x:np.array([x['sequence'][i-self.half_ck:i+self.half_ck+1] for i in x['Ordinary_sites']]),axis=1);
        #过滤所有不符合要求的seqs
        seq_df['Donor_seqs']=seq_df['Donor_seqs'].progress_apply(lambda x:self.filter_base(x));
        seq_df['Accept_seqs']=seq_df['Accept_seqs'].progress_apply(lambda x:self.filter_base(x));
        seq_df['Ordinary_seqs']=seq_df['Ordinary_seqs'].progress_apply(lambda x:self.filter_base(x));
        #将序列编码 方便后续模型操作
        seq_df['Donor_code']=seq_df.progress_apply(lambda x: np.array([self.feature_code(i) for i in x['Donor_seqs']]),axis=1);
        seq_df['Accept_code']=seq_df.progress_apply(lambda x: np.array([self.feature_code(i) for i in x['Accept_seqs']]),axis=1);
        seq_df['Ordinary_code']=seq_df.progress_apply(lambda x: np.array([self.feature_code(i) for i in x['Ordinary_seqs']]),axis=1);
        #将seq_df 本身赋值于自身
        self.df=seq_df;
        return seq_df;
    # to save sequence_df
    def to_save(self,path):
        if self.df.empty:
            print("nothing to save,please run object.feature_df first")
            return;
        else:
            data=pd.DataFrame();
            concat=np.concatenate
            data_Doseq=concat(self.df['Donor_seqs']);
            data_Acseq=concat(self.df['Accept_seqs']);
            data_Orseq=concat(self.df['Ordinary_seqs']);
            choose_index=concat((np.ones(int(1e5),dtype=int),np.zeros(len(data_Orseq)-int(1e5),dtype=int))).astype(bool);
            np.random.shuffle(choose_index);
            data['seqs']=concat((data_Doseq,data_Acseq,data_Orseq[choose_index]));
            data_Docode=concat(self.df['Donor_code']);
            data_Accode=concat(self.df['Accept_code']);
            data_Orcode=concat(self.df['Ordinary_code']);
            data['seqs_code']=concat((data_Docode,data_Accode,data_Orcode[choose_index]),axis=0).tolist();
            data_nums=data.shape[0];
            doseqs_nums=data_Doseq.shape[0];
            acseqs_nums=data_Acseq.shape[0];
            data['IsDonor']=concat((np.ones(doseqs_nums,dtype=int),np.zeros(data_nums-doseqs_nums,dtype=int)));
            data['IsAcceptor']=concat((np.zeros(doseqs_nums),np.ones(acseqs_nums),np.zeros(data_nums-doseqs_nums-acseqs_nums)));
            #保存到硬盘
            file=os.path.split(path)[0];
            if not os.path.isdir(file):
                os.mkdir(file);
            data[0:10000].to_csv(path);
            
            
            
            
            
            
            
            
        

In [3]:
#定义滑动窗口
chunk_size=11;
#生成训练集的特征df
training_model=sequence_df(training_set_path,chunk_size)
training_df=training_model.feature_df();
testing_model=sequence_df(testing_set_path,chunk_size)
testing_df=testing_model.feature_df();

processing: 100%|██████████| 570/570 [00:22<00:00, 24.81it/s]


In [5]:
training_path="../data/data_ready/training_set.csv"
testing_path="../data/data_ready/testing_set.csv"
training_model.to_save(training_path);
testing_model.to_save(testing_path);

In [21]:
traning_df=feature_process(traning_df);


         ID                                                ADS  \
1  AB002059  9106..9239,9843..9993,11889..11960,16575..1665...   
2  AB005803  2301..2483,5205..5321,6208..6298,7892..8058,90...   
3  AB005990  1..195,845..1035,1543..1745,1831..2031,2116..2...   
4  AB009589                            8540..9479,10624..10949   

                                            sequence  \
1  ggtgaaacctcatctctactaaaaatacaaaaaattggtcaggcgt...   
2  catctgaggccactctctagttcccataatgctaataggaaccatc...   
3  atgacccagaccctcaagtacgcctccagagtgttccatcgcgtcc...   
4  ccaatcagtttaaattttctagacatttttattctgtctgctctag...   

                                        Accept_sites  \
1  [9106, 9843, 11889, 16575, 16841, 17017, 17174...   
2       [2301, 5205, 6208, 7892, 9055, 11357, 13312]   
3  [1, 845, 1543, 1831, 2116, 2492, 2866, 3234, 3...   
4                                      [8540, 10624]   

                                         Donor_sites  
1  [9239, 9993, 11960, 16650, 16934, 17097, 17315...

  Using cached functools-0.5.tar.gz (4.9 kB)
    ERROR: Command errored out with exit status 1:
     command: /root/anaconda3/envs/datamine/bin/python3.6 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-evk31i_2/functools_5a7a61a6b4aa4ce5a30928ac157edbb8/setup.py'"'"'; __file__='"'"'/tmp/pip-install-evk31i_2/functools_5a7a61a6b4aa4ce5a30928ac157edbb8/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-gnon_xg4
         cwd: /tmp/pip-install-evk31i_2/functools_5a7a61a6b4aa4ce5a30928ac157edbb8/
    Complete output (16 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/root/anaconda3/envs/datamine/lib/python3.6/site-packages/setuptools/__init__.py", line 3, in <module>
        from fnmatch import fnmatchcase
      File "/root/anaconda3/envs/datamine/l